In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import datetime
import imageio as img
import os

In [2]:
nc_file = nc.Dataset('/localdata/cases/20180519/GLM_data/GLM-00-00_20180520_010000_60_1src_056urad-dx_group_extent.nc', 'r')
nc_file

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): nx(3249), ny(1300), ntimes(1)
    variables(dimensions): int32 goes_imager_projection(), float32 x(nx), float32 y(ny), float32 time(ntimes), float32 group_extent_density(ntimes,nx,ny)
    groups: 

# Importing and preparing the GLM Data

In [3]:
#Constants
#NOTE: Can only do three hours at a time

#Used for GLM timesteps/filenames
start_y = '2018'                                        #Format: YYYY(UTC-Relative)
start_m = '05'                                          #Format: MM(UTC-Relative)
start_d =  '20'                                         #Format: DD(UTC-Relative)
start_h = '02'                                          #Format: HH (UTC-Relative)
start_M = '10'                                          #Format: MM (UTC-Relative)

end_y = '2018'                                          #Format: YYYY(UTC-Relative)
end_m = '05'                                            #Format: MM(UTC-Relative)
end_d =  '20'                                           #Format: DD(UTC-Relative)
end_h = '07'                                            #Format: HH (UTC-Relative)
end_M = '00'                                            #Format: MM (UTC-Relative)

#GLM Data Info
GLM_file_location = '/localdata/cases/20180519/GLM_data/' #Location of the data that you are pulling from
GLM_name_start = 'GLM-00-00_'                          #First part of the file name before the time
GLM_name_end = '_60_1src_056urad-dx_'                  #Last part of the file name after the time
GLM_var_name = 'group_extent'                               #Variable that is used in the file name
GLM_data_name = 'group_extent_density'                      #Variable that is used to pull the data from the netCDF file
GLM_full_name = 'Group Extent Density'
GLM_units = ' (Groups per Min.)'
file_type = '.nc'                                       #Datatype for netCDF file

#MRMS Data Info
MRMS_file_loc = '/localdata/cases/20180519/merged_radar/merged/'
MRMS_product1 = 'MESH/00.25/'
MRMS_product2 = 'Reflectivity_0C/00.25/'
MRMS_var1_name = 'MESH'
MRMS_var1_units = ' (mm)'
MRMS_var2_name = 'Reflectivity_0C'
MRMS_contour_val = 40
spacing = 0.01 #MRMS spacing between points
data_crs = ccrs.PlateCarree()

save_file = MRMS_var1_name+'_VS_'+GLM_data_name
save_location = '/localdata/cases/20180519/GLM_MRMS_pics/animations/'+save_file+'/'        #a premade folder to save the data to

In [4]:
#Setting the start and end times
start_time = start_y + start_m + start_d + start_h + start_M                       
end_time = end_y + end_m + end_d + end_h + end_M

#Creating a list of start and end times to use for the loop and for plotting
time = pd.PeriodIndex(start = start_time, end = end_time, freq='T')
time_ymd = time.strftime('%Y%m%d')
time_hms = time.strftime('%H%M%S')
time_hm = time.strftime('%H%M')

time_Y = time.strftime('%Y')
time_m = time.strftime('%m') #month
time_D = time.strftime('%D')
time_H = time.strftime('%H')
time_M = time.strftime('%M') #Minute
time_S = time.strftime('%S')

#Used for the timeseries
timeseries = time.to_timestamp()

In [5]:
#Creating an array of file_names using the start_time and end_time
i = 0
GLM_file_names = np.empty([0])
while i < len(time):
    name = GLM_file_location + GLM_name_start + time_ymd[i] + '_' + time_hms[i]+ GLM_name_end + GLM_var_name + file_type
    GLM_file_names = np.append(GLM_file_names, name)
    i += 1

In [6]:
#Setting the locations to pull the data from (use Basic_Plot_GLM.ipynb to get bounds)
x_locs = np.arange(1600,1851,1)
y_locs = np.arange(740,811,1)

In [7]:
#Pulling the maximum value from the GLM variable per timestep 
k = 0
max_GLM_data = np.empty([0])
while k < len(GLM_file_names):
    nc_file_GLM = nc.Dataset(GLM_file_names[k],'r') #Loading in the GLM Data
    GLM_var = nc_file_GLM.variables[GLM_data_name][:,:]
    GLM_var = np.squeeze(GLM_var) #Reducing the dimensionality to 2D
    GLM_var[GLM_var==0] = np.nan #Removing the zeros (to NaN's)
    
    x = nc_file_GLM.variables['x'][:] #Getting the x points for the data
    y = nc_file_GLM.variables['y'][:] #Getting the y points for the data
    
    GLM_iso_var = np.ones((len(x),len(y)))*np.nan
    
    for i in x_locs: #Loop to fill GLM_iso_var with only the data in the predefined bounds
        for j in y_locs:
            GLM_iso_var[i,j] = GLM_var[i,j]
    
    max_GLM_data = np.append(max_GLM_data, np.nanmax(GLM_iso_var))
    k += 1
print (np.nanmax(max_GLM_data))

/localdata/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: All-NaN slice encountered


208.00143432617188


In [13]:
nc_file_GLM.variables['time'].units

'seconds since 2018-05-20 00:00:00'

In [8]:
bounds = np.arange(0, 210, 10)                            #Bounds for the colorbar

# Importing and preparing the MRMS data (var1 and var2)

In [9]:
#Gets the maximum value from each netCDF file in the folder and the timeseries data
max_MRMS_data = np.empty([0])
time_MRMS = np.empty([0])
MRMS_filename_list = np.empty([0])
MRMS_hm = np.empty([0])

for MRMS_filename in os.listdir(MRMS_file_loc+MRMS_product1):
    nc_file_MRMS = nc.Dataset(MRMS_file_loc+MRMS_product1+MRMS_filename, 'r')
    MRMS_filename_list = np.append(MRMS_filename_list, MRMS_filename)#Getting the list of MRMS Filenames
    #makes the variable from the netCDF file into a local variable that we can use when plotting data
    MRMS_var1 = nc_file_MRMS.variables[MRMS_var1_name][:,:]
    max_MRMS_data = np.append(max_MRMS_data, np.nanmax(MRMS_var1)) #Pulling the maximum values in the MRMS data
    
    dtime = datetime.datetime.utcfromtimestamp(nc_file_MRMS.Time) #Timesteps to use for the timeseries
    MRMS_hm = np.append(MRMS_hm, dtime.strftime('%H%M')) #List of times to use for the plot loop
    time_MRMS = np.append(time_MRMS, dtime)

In [10]:
nc_file_MRMS

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    TypeName: MESH
    DataType: LatLonGrid
    Latitude: 41.49999999999999
    Longitude: -95.91
    Height: 250.0
    Time: 1526800044
    FractionalTime: 0.22599999999874854
    attributes:  Unit
    Unit-unit: dimensionless
    Unit-value: mm
    LatGridSpacing: 0.01
    LonGridSpacing: 0.01
    MissingData: -99900.0
    RangeFolded: -99901.0
    dimensions(sizes): Lat(180), Lon(455)
    variables(dimensions): float32 MESH(Lat,Lon)
    groups: 

In [51]:
#Loading in the MRMS Data (variable 2)
nc_file_MRMS = nc.Dataset(MRMS_file_loc+MRMS_product2+'20180520-021046.netcdf','r')

u_lat = nc_file_MRMS.Latitude #UPPER latitude
l_lon = nc_file_MRMS.Longitude #LEFT Longitude

#makes the variable from the netCDF file into a local variable that we can use when plotting data
MRMS_var2 = nc_file_MRMS.variables[MRMS_var2_name][:,:]

#Getting the dimensions of x and y for plotting
len_x = len(MRMS_var2[0,:])
len_y = len(MRMS_var2[:,0])

#Setting array for x and y
x_MRMS = np.arange(l_lon, l_lon+(len_x*spacing), spacing) #W->E
y_MRMS = np.arange(u_lat, u_lat-(len_y*spacing), -spacing) #Because of plotting direction, latitudes are N->S

# Making the images/animation

In [52]:
less_locs = np.where(MRMS_hm < time_hm[0])[0]
i = less_locs[-1] 
nc_file_MRMS = nc.Dataset(MRMS_file_loc + MRMS_product2 + MRMS_filename_list[i],'r')
MRMS_var_true = nc_file_MRMS.variables[MRMS_var2_name][:,:]
MRMS_var_true[MRMS_var_true<MRMS_contour_val] = False
MRMS_var_true[MRMS_var_true>=MRMS_contour_val] = True


m = 0
length = len(time)

while m < length:
    #GLM DATA being read in
    nc_file_GLM = nc.Dataset(GLM_file_names[m], 'r')
    GLM_var = nc_file_GLM.variables[GLM_data_name][:,:]
    GLM_var = np.squeeze(GLM_var)
    GLM_var[GLM_var==0] = np.nan
    #Checking if the times match so the MRMS contour-data can advance
    if time_hm[m] == MRMS_hm[i+1]:
        i+=1
        nc_file_MRMS = nc.Dataset(MRMS_file_loc + MRMS_product2 + MRMS_filename_list[i],'r')
        MRMS_var_true = nc_file_MRMS.variables[MRMS_var2_name][:,:]
        MRMS_var_true[MRMS_var_true<25] = False
        MRMS_var_true[MRMS_var_true>=25] = True
    #Creating plot with all the features
    fig = plt.figure(figsize=(15, 15))
    ax1 = plt.subplot(2, 1, 1, projection=ccrs.Geostationary(-75, 35786023.0))

    #Code to create the map in the upper panel
    ax1.set_extent([x_MRMS[0], x_MRMS[-1], y_MRMS[-1], y_MRMS[0]])
    one = ax1.contourf(x * 35786023.0, y * 35786023.0, GLM_var.T, levels=bounds, cmap=plt.get_cmap('nipy_spectral'), zorder=10)
    ax1.contour(x_MRMS, y_MRMS, MRMS_var_true, transform=data_crs, colors='k', zorder=11, linewidths=1.)
    plt.title('GLM ' + GLM_full_name + ' (0C dBZ = 40)   ' + time_H[m] + ':' + time_M[m] + '   ' + time_D[m], fontsize='xx-large', loc='left')
    ax1.coastlines(resolution='110m')
    ax1.add_feature(cfeature.LAND)
    ax1.add_feature(cfeature.OCEAN)
    ax1.add_feature(cfeature.STATES, zorder=8, linestyle=':')
    ax1.add_feature(cfeature.BORDERS, zorder=9)
    ax1.add_feature(cfeature.LAKES)
    ax1.add_feature(cfeature.RIVERS)
    cbar_one = plt.colorbar(one)
    cbar_one.set_label(GLM_full_name+GLM_units, fontsize='large')
    
    #Code to create the timeseries in the lower panel
    ax2 = plt.subplot(2, 1, 2)
    ax3 = ax2.twinx()
    ax2.plot(timeseries, max_GLM_data, color='b', zorder=5, label='Maximum '+GLM_full_name)
    ax3.plot(time_MRMS, max_MRMS_data, color='k', label='Maximum '+MRMS_var1_name)
    ax2.grid(True, linestyle=':', zorder=1)
    ax2.axvline(pd.Timestamp('2018-05-20 04:54'), color='r', linewidth='1', zorder=2, label='Sig Hail Report')
    ax2.axvline(pd.Timestamp('2018-05-20 05:12'), color='r', linewidth='1', zorder=3)
    ax2.axvline(pd.Timestamp('2018-05-20 05:15'), color='r', linewidth='1', zorder=4)
    ax2.axvline(timeseries[m], color='k', linewidth='3', zorder=6)   
    ax2.set_ylim(0,)
    ax3.set_ylim(0,)
    ax2.set_xlim(pd.Timestamp(timeseries[0]),pd.Timestamp(timeseries[-1]))
    plt.title('Maximum '+GLM_full_name+' vs. Maximum '+MRMS_var1_name+' (Sig-Hail Report Times)', fontsize='xx-large', loc='left')
    plt.title(str(round(max_GLM_data[m])) + GLM_units, fontsize='xx-large', loc='right')
    ax2.set_xlabel('Time (UTC)', fontsize='large')
    ax2.set_ylabel(GLM_full_name+GLM_units, fontsize='large')
    ax3.set_ylabel(MRMS_var1_name + MRMS_var1_units, fontsize='large')
    ax2.legend(loc='upper left',shadow=True)
    ax3.legend(loc='upper right',shadow=True)
    
    plt.savefig(save_location + save_file + '_' + time_hms[m] + '.png', orientation="landscape", format="png")
    plt.close()
    print (time_hm[m])
    m += 1
    

0210
0211
0212
0213
0214
0215
0216
0217
0218
0219
0220
0221
0222
0223
0224
0225
0226
0227
0228
0229
0230
0231
0232
0233
0234
0235
0236
0237
0238
0239
0240
0241
0242
0243
0244
0245
0246
0247
0248
0249
0250
0251
0252
0253
0254
0255
0256
0257
0258
0259
0300
0301
0302
0303
0304
0305
0306
0307
0308
0309
0310
0311
0312
0313
0314
0315
0316
0317
0318
0319
0320
0321
0322
0323
0324
0325
0326
0327
0328
0329
0330
0331
0332
0333
0334
0335
0336
0337
0338
0339
0340
0341
0342
0343
0344
0345
0346
0347
0348
0349
0350
0351
0352
0353
0354
0355
0356
0357
0358
0359
0400
0401
0402
0403
0404
0405
0406
0407
0408
0409
0410
0411
0412
0413
0414
0415
0416
0417
0418
0419
0420
0421
0422
0423
0424
0425
0426
0427
0428
0429
0430
0431
0432
0433
0434
0435
0436
0437
0438
0439
0440
0441
0442
0443
0444
0445
0446
0447
0448
0449
0450
0451
0452
0453
0454
0455
0456
0457
0458
0459
0500
0501
0502
0503
0504
0505
0506
0507
0508
0509
0510
0511
0512
0513
0514
0515
0516
0517
0518
0519
0520
0521
0522
0523
0524
0525
0526
0527
0528
0529


In [53]:
#Procedure for creating GIFs from the data

file_names = []
for i in time_hms: #FOR loop for creating a list of file_names of the images we created
    file_names.append(save_location + save_file + '_' + i + '.png')

images = []    
for j in file_names: #FOR loop for reading in all of the images and making them into a GIF
    images.append(img.imread(j))
img.mimsave(save_location + save_file +'.gif', images, format='gif', loop=0, duration=.1, subrectangles=True)

In [ ]:
#img.help(name='gif')